# In this notebook, I do some exploration of SeqIO from the Biopython package. The function I plan to use with DeepFRI can be found in this notebook.
## Author: Korede Ogundele
## Date Created: March 4, 2024
## Last Modified: March 22, 2024

In [1]:
from Bio import SeqIO
import os

### Print descriptions of NACHT; 6 isoforms of one protein.

In [2]:
# read file and print descriptions for each isoform
for isoform in SeqIO.parse("nacht.fasta", "fasta"):
    print(isoform.description)

sp|Q96P20|NLRP3_HUMAN NACHT, LRR and PYD domains-containing protein 3 OS=Homo sapiens OX=9606 GN=NLRP3 PE=1 SV=3
sp|Q96P20-2|NLRP3_HUMAN Isoform 1 of NACHT, LRR and PYD domains-containing protein 3 OS=Homo sapiens OX=9606 GN=NLRP3
sp|Q96P20-3|NLRP3_HUMAN Isoform 3 of NACHT, LRR and PYD domains-containing protein 3 OS=Homo sapiens OX=9606 GN=NLRP3
sp|Q96P20-4|NLRP3_HUMAN Isoform 4 of NACHT, LRR and PYD domains-containing protein 3 OS=Homo sapiens OX=9606 GN=NLRP3
sp|Q96P20-5|NLRP3_HUMAN Isoform 5 of NACHT, LRR and PYD domains-containing protein 3 OS=Homo sapiens OX=9606 GN=NLRP3
sp|Q96P20-6|NLRP3_HUMAN Isoform 6 of NACHT, LRR and PYD domains-containing protein 3 OS=Homo sapiens OX=9606 GN=NLRP3


Each record is a SeqRecord, each with the following attributes: 
'annotations',
 'dbxrefs',
 'description',
 'features',
 'format',
 'id',
 'letter_annotations',
 'lower',
 'name',
 'reverse_complement',
 'seq',
 'translate',
 'upper'


 
 ### We can write the output to a file using the SeqIO write function.

In [3]:
with open("output.fasta", "w") as output_file:
    SeqIO.write(isoform, output_file, "fasta")

In [10]:
isoform.format

<bound method SeqRecord.format of SeqRecord(seq=Seq('MKMASTRCKLARYLEDLEDVDLKKFKMHLEDYPPQKGCIPLPRGQTEKADHVDL...PSW'), id='sp|Q96P20-6|NLRP3_HUMAN', name='sp|Q96P20-6|NLRP3_HUMAN', description='sp|Q96P20-6|NLRP3_HUMAN Isoform 6 of NACHT, LRR and PYD domains-containing protein 3 OS=Homo sapiens OX=9606 GN=NLRP3', dbxrefs=[])>

## Get a set of all protein names from UniProt file with all human protein sequences.

In [4]:
#create empty set
protein_set = set()

# read file and append descriptions for each protein to set
for protein in SeqIO.parse("uniprotkb_proteome_human.fasta", "fasta"):
    # splice string to only insert protein name (not name with isoform code) to set
    protein_set.add(protein.name.split("|")[2])

### UniProtKB gave 82,485 human proteins. If the set has been constructed properly, it should have this exact number of proteins in it.

In [22]:
len(protein_set)

82485

Success.

## Function to select certain proteins from a fasta file and output to another fasta file
This will be used to select each protein from the set and print its fasta sequences to an individual file (filename being protein name).

In [40]:
def filter_sequences(input_filename, desired_protein):
    """
    This function reads through a file of many FASTA sequences and selects those for a particular protein.
    The sequences are then written and stored in an output file.

    Arguments
    ---------
    input_filename : str
        name of the input FASTA file.
    output_filename : str
        name of the output FASTA file.
    desired_proteins : list
        list of proteins to keep and write to output file

    Returns
    -------
    *.fasta : fasta file
        fasta file containing desired protein sequences. The file will bear protein name.
    """
    instances = []

    # output file will have the name of the protein and be moved to individual_fastas folder
    # CHANGE FILE PATH ACCORDINGLY
    output_filename = f"individual_fastas/{desired_protein}"
    
    
    # check if the output file already exists from previous run
    if os.path.exists(output_filename):
        print(f"Output file '{output_filename}' already exists. Skipping '{desired_protein}'")
        return

    for record in SeqIO.parse(input_filename, "fasta"):
        # check if the protein sequence has the exact desired name
        #protein_name = protein.name.split("|")[2]
        if desired_protein == record.name.split("|")[2]:
            # if so, add record to instance list
            instances.append(record)
    
    
    # write desired sequences to the output file
    with open(output_filename, "w") as output_file:
        SeqIO.write(instances, output_file, "fasta")


# run the function on the whole set of proteins. 
for protein in protein_set:
    print(f"generating fasta file for {protein}")
    input_filename = "uniprotkb_proteome_human.fasta"
    protein_to_select = protein
    filter_sequences(input_filename, protein_to_select)

### Let's try with a small subset of proteins from protein_set first.

In [43]:
mini_list = list(protein_set)[:10]


for protein in mini_list:
    input_filename = "uniprotkb_proteome_human.fasta"
    protein_to_select = protein
    filter_sequences(input_filename, protein_to_select)

### Success! Now we run the function on the whole set of proteins. Outputs will be moved the file individual_fastas, which will not be pushed to the respository.

In [44]:
for protein in protein_set:
    input_filename = "uniprotkb_proteome_human.fasta"
    protein_to_select = protein
    filter_sequences(input_filename, protein_to_select)